In [1]:
def read_number(line, index):
    number = 0
    while index < len(line) and line[index].isdigit():
        number = number * 10 + int(line[index])
        index += 1
    if index < len(line) and line[index] == '.':
        index += 1
        decimal = 0.1
        while index < len(line) and line[index].isdigit():
            number += int(line[index]) * decimal
            decimal /= 10
            index += 1
    token = {'type': 'NUMBER', 'number': number}
    return token, index


def read_plus(line, index):
    token = {'type': 'PLUS'}
    return token, index + 1


def read_minus(line, index):
    token = {'type': 'MINUS'}
    return token, index + 1

def read_multiply(line, index):
    token = {'type':'MULTIPLY'}
    return token, index + 1

def read_divide(line, index):
    token = {'type':'DIVIDE'}
    return token, index + 1

def read_left_bracket(line, index):
    token = {'type':'LEFT_BRACKET'}
    return token, index + 1
def read_right_bracket(line, index):
    token = {'type':'RIGHT_BRACKET'}
    return token, index + 1

def tokenize(line):
    tokens = []
    index = 0
    while index < len(line):
        if line[index].isdigit():
            (token, index) = read_number(line, index)
        elif line[index] == '+':
            (token, index) = read_plus(line, index)
        elif line[index] == '-':
            (token, index) = read_minus(line, index)
        elif line[index] == '*':
            (token, index) = read_multiply(line, index)
        elif line[index] == '/':
            (token, index ) = read_divide(line, index)
        elif line[index] == '(':
            (token, index) = read_left_bracket(line, index)
            index +=1
        elif line[index] == ')':
            (token, index) = read_right_bracket(line, index) 
            index +=1
        else:
            print('Invalid character found: ' , line[index])
            index +=1
            continue
            
        tokens.append(token)
    return tokens

def evaluate_brackets(tokens):    #Thoughts: 1st: delete the bracket.  2nd:calculate mul&div   3rd: calculate pluss&minus   4th:find the result and reserve it as a number
                                    #how can I find the right bracket here 
    index = 0
    while index < len(tokens):
        if tokens[index]['type'] == 'LEFT_BRACKET':
           
            left = index
            right = left
            while right < len(tokens):
                if tokens[right]['type'] == 'RIGHT_BRACKET':
                    break
                right += 1

           
            inner_tokens = tokens[left + 1 : right]       # 取中间部分（不包括括号）
      
            result = evaluate(inner_tokens)      # 递归调用 evaluate
           
            new_token = {'type': 'NUMBER', 'number': result}   # 构造新 token
        
            tokens = tokens[:left] + [new_token] + tokens[right + 1:]    # 替换原来的一段：左括号 到 右括号

            index = 0    # 从头重新扫描（因为 tokens 长度变了）
            
            continue

        index += 1

    return tokens

def evaluate_mul_div(tokens):
    
    mid = []               #calculate multiply and divide 1st and temporaraliy reserve here
    index = 0
    
    while index < len(tokens):

        if tokens[index]['type'] == 'NUMBER':
            if mid and mid[-1]['type'] == 'MULTIPLY':
                mid.pop()                                       
                if mid and mid[-1]['type'] == 'NUMBER':
                    
                    left = mid.pop()['number']
                    result = left * tokens[index]['number']
                    mid.append({'type':'NUMBER','number':result})
                else:
                    print("No number") 
                    exit(1)

            elif mid and mid[-1]['type'] == 'DIVIDE':
                mid.pop()
                if mid and mid[-1]['type'] == 'NUMBER':
                    left = mid.pop()['number']
                    result = left / tokens[index]['number']
                    mid.append({'type':'NUMBER','number':result})
                else:
                    print("No number")
                    exit(1)

            else:
                mid.append(tokens[index])
        else:
            mid.append(tokens[index])   # 把 PLUS / MINUS 收进 mid
        
        index += 1
    return mid

# divide def  (15-20 lines)
def evaluate_plus_minus(tokens):
    
    answer = 0
    tokens = [{'type':'PLUS'}] + tokens     ##copy and pastse from the sample function
    index = 1
    
    while index < len(tokens):
        if tokens[index]['type'] == 'NUMBER':
            if tokens[index - 1]['type'] == 'PLUS':
                answer += tokens[index]['number']
            elif tokens[index - 1]['type'] == 'MINUS':
                answer -= tokens[index]['number']
            else:
                print('Invalid syntax')
                exit(1)
                
        index += 1
    return answer
      
def evaluate(tokens):
    first = evaluate_brackets(tokens)     #括号外面有乘号怎么办
    medium = evaluate_mul_div(first)
    final_result = evaluate_plus_minus(medium)
    return final_result

def test(line):
    tokens = tokenize(line)
    actual_answer = evaluate(tokens)
    expected_answer = eval(line)
    if abs(actual_answer - expected_answer) < 1e-8:
        print("PASS! (%s = %f)" % (line, expected_answer))
    else:
        print("FAIL! (%s should be %f but was %f)" % (line, expected_answer, actual_answer))




In [ ]:
def run_test():
    print("===Test Start===")
    test("1+(2*3)+1")
    test("0.3 + (12-9*7) - (21.2 + 1/9 + 1) - 0.5")
    print("===Test Finished===")
run_test()